In [1]:
from regelum.scenario import MPCTorch
from regelum.simulator import CasADi
import numpy as np
from tqdm.notebook import tqdm

import sys
sys.path.append('../')

from src.system_temp import HydraulicSystem
# from src.observer_temp import HydraulicObserver
# from src.policy import PDController

In [2]:
from regelum.policy import Policy
from regelum.utils import rg
from regelum.model import ModelWeightContainer
from regelum.system import ComposedSystem, System
from regelum.optimizable.core.configs import CasadiOptimizerConfig
from numpy import ndarray
from regelum.model import Model, ModelNN, ModelQuadLin
from regelum.optimizable.core.configs import OptimizerConfig
from regelum.objective import mpc_objective
from regelum.predictor import EulerPredictor
from regelum.objective import RunningObjective
from regelum.data_buffers import DataBuffer

In [3]:
from regelum.callback import (
    ScenarioStepLogger,
    # StateTracker,
    # HistoricalDataCallback,
)
from src.callback import HistoricalDataCallback, SimulatorStepLogger

from regelum import set_ipython_env
%matplotlib inline
callbacks = [SimulatorStepLogger, ScenarioStepLogger, HistoricalDataCallback]
ScenarioStepLogger.cooldown = 0.01
callbacks = set_ipython_env(callbacks=callbacks, interactive=True)

In [4]:
from regelum.model import ModelQuadLin

# from regelum.typing import RgArray
# from regelum.objective import Objective

# class HydraulicObjective(Objective):
#     def __init__(
#         self,
#         system: HydraulicSystemModel
#     ):
#         self.l_crit = system._parameters["l_crit"]
    
#     def __call__(
#         self, 
#         observation: RgArray, 
#         action: RgArray,
#         is_save_batch_format = False,
#     ):
#         # return rg.array(
#         #     [[(observation[:,0] - self.l_crit)**2]],
#         #     prototype=observation
#         # )
        
#         result = (observation[:,0] - self.l_crit)**2
        
#         return result[None, :]
        
#         # return rg.array(
#         #     [result],
#         #     prototype=observation
#         # )

class HydraulicObjectiveModel(ModelQuadLin):
    
    def __init__(
        self,
        system: HydraulicSystem,
        *args,
        **kwargs
    ):
        super().__init__(*args, **kwargs)
        
        # self.l_crit = system._parameters["l_crit"]
    
    def forward(self, inputs, weights=None):
        if weights is None:
            weights = self.weights
            quad_matrix = self._quad_matrix
            linear_coefs = self._linear_coefs
            # if isinstance(inputs, torch.Tensor):
            #     quad_matrix = torch.FloatTensor(quad_matrix).to(inputs.device)
            #     if linear_coefs is not None:
            #         linear_coefs = torch.FloatTensor(linear_coefs).to(inputs.device)
        else:
            quad_matrix, linear_coefs = self.get_quad_lin(weights)

        substract = rg.zeros(
            inputs.shape,
            prototype=inputs
        )
        
        # substract[:,0] = self.l_crit
        
        # inputs -= substract

        return ModelQuadLin.quadratic_linear_form(
            inputs,
            quad_matrix,
            linear_coefs,
        )

In [5]:
MAX_STEP = 1e-7
PREDICTION_HORIZON = 2
FINAL_TIME = 10e-3
SAMPLING_TIME = 1e-3
ACTION_SIZE = 1

pred_step_size = SAMPLING_TIME
# pred_step_size = 3 * MAX_STEP

# Define the initial state (initial position of the kinematic point).
p_atm = 1e5
initial_state = rg.array([1e3, 0, 0, 1.15*p_atm, 1.01*p_atm])

model = ModelWeightContainer(
    dim_output=ACTION_SIZE, 
    weights_init=rg.zeros((PREDICTION_HORIZON + 1, ACTION_SIZE), rc_type=rg.CASADI)
)  # A trivial model contains our predicted actions
# system = HydraulicSystem(
#     init_state=initial_state
# )
from src.system_temp import HydraulicSystemReference

system = HydraulicSystemReference(
    init_state=initial_state
)

# system_model = HydraulicSystem(
#     init_state=initial_state
# )
# observer = HydraulicObserver(
#     system=system_model,
# )

from regelum.predictor import EulerPredictorMultistep
predictor = EulerPredictorMultistep(
    system=system, pred_step_size=pred_step_size,
    n_steps=10
)  # Predictor is used for MPC-prediction
# RUNNING OBJECTIVE
running_objective_model = HydraulicObjectiveModel(
    system=system,
    quad_matrix_type="diagonal", weights=[1, 0, 0, 0, 0, 0]
)  # A quadratic model for running (a.k.a. instant) objective. We penalize our agent for x and y
# running_objective_model = ObjectiveModel(
#     system=system_model,
# )
running_objective = RunningObjective(
    model=running_objective_model
)  # Wrap `running_objective_model` into `RunningObjective` class
# running_objective = HydraulicObjective(
#     system=system_model,
# )  # Wrap `running_objective_model` into `RunningObjective` class

class MPC(Policy):

    def __init__(
        self,
        model: Model | ModelNN = model,
        system: System | ComposedSystem = system,
        action_bounds: list | ndarray | None = system.action_bounds,
        optimizer_config: OptimizerConfig | None = CasadiOptimizerConfig(),
        discount_factor: float | None = 1,
        epsilon_random_parameter: float | None = None,
        prediction_horizon: int = PREDICTION_HORIZON,
        running_objective: RunningObjective = running_objective,
        predictor: EulerPredictor = predictor,
    ):
        """Instantiate MPC policy."""
        super().__init__(
            model,
            system,
            action_bounds,
            optimizer_config,
            discount_factor,
            epsilon_random_parameter,
        )  # Initialize the parent `Policy` class.

        self.prediction_horizon = prediction_horizon
        self.running_objective = running_objective
        self.predictor = predictor
        ###### Define the optimization problem
        self.observation_var = self.create_variable(
            1,  # dimensionality of axis 0
            self.system.dim_observation,  # dimensionality of axis 1
            name="observation",
            is_constant=True,  # is_constant set to `True` as `observation` is a constant parameter of optimization
        )
        self.est_state_var = self.create_variable(
            1,  # dimensionality of axis 0
            self.system.dim_state,  # dimensionality of axis 1
            name="estimated_state",
            is_constant=True,  # is_constant is set to `True` as `estimated_state` is a constant parameter of optimization
        )
        self.policy_model_weights_var = self.create_variable(
            name="policy_model_weights",
            is_constant=False,  # is_constant is set to False because policy_model_weights is a decision variable in our optimization problem
            like=self.model.named_parameters,  # like parameter utilizes the dimensions of the model's weights for compatibility
        )
        ## Let us register bounds for policy model weights to be within action bounds
        (
            self.action_bounds_tiled,
            self.action_initial_guess,
            self.action_min,
            self.action_max,
        ) = self.handle_bounds(
            self.action_bounds,
            self.dim_action,
            tile_parameter=self.model.weights.shape[0],
        )
        self.register_bounds(self.policy_model_weights_var, self.action_bounds_tiled)

        ## Make `Optimizable` aware of objective function and variables it depends on
        self.register_objective(
            self.cost,
            variables=[
                self.observation_var,
                self.est_state_var,
                self.policy_model_weights_var,
            ],
        )

    def optimize(self, databuffer: DataBuffer) -> None:
        """Define optimization routine for `Optimizable` class."""
        new_weights = super().optimize_symbolic(
            **databuffer.get_optimization_kwargs(
                keys=["observation", "estimated_state"],
                optimizer_config=self.optimizer_config,
            ),
            policy_model_weights=self.policy_model_weights_var(),
        )[
            "policy_model_weights"
        ]  # Get the optimized weights from `Optimizable` class
        self.model.update_and_cache_weights(new_weights)

    def cost(self, observation, estimated_state, policy_model_weights):
        """Cost function for MPC Policy."""
        return mpc_objective(
            observation=observation,
            estimated_state=estimated_state,
            policy_model_weights=policy_model_weights,
            discount_factor=self.discount_factor,
            running_objective=self.running_objective,
            prediction_horizon=self.prediction_horizon,
            predictor=self.predictor,
            model=self.model,
        )  # Call `mpc_objective` function to get the cost of current state and sequence of predicted actions

In [6]:
system._parameters["l_crit"]

1.9167193623596763

In [7]:
D_work_exit_2_ratio = system._parameters["D_work_exit_2_ratio"]

In [8]:
from regelum.observer import ObserverReference

observer = ObserverReference(
    reference=np.array([
        1000.5218268464024,
        0,
        0,
        1e5,
        1e5,
    ])
)

In [9]:
from regelum.critic import CriticTrivial
from regelum.scenario import RLScenario
from regelum.event import Event

# initial_state = rg.array([-10, 10])

simulator = CasADi(
    system=system,
    state_init=initial_state,
    action_init=rg.array([0]),
    time_final=FINAL_TIME,
    max_step=MAX_STEP,
    # first_step=1e-6,
    # atol=1e-5,
    # rtol=1e-3,
)

# scenario = RLScenario(
#     policy=MPC(),
#     # observer = observer,
#     critic=CriticTrivial(),  # mocked critic
#     running_objective=running_objective,
#     policy_optimization_event=Event.compute_action,  # run policy optimization on each step
#     discount_factor=1.0,
#     sampling_time=SAMPLING_TIME,
#     simulator=simulator,
#     N_episodes=1,
#     N_iterations=1,
# )

scenario = MPCTorch(
    running_objective=RunningObjective(model=ModelQuadLin(
        quad_matrix_type='diagonal',
        weights=np.array([10., 0., 0., 0., 0., 0.])
    )),
    simulator=simulator,
    observer=observer,
    prediction_horizon=PREDICTION_HORIZON,
    sampling_time=SAMPLING_TIME,
    n_epochs=100,
    opt_method_kwargs={'lr': 1e0},
    predictor=predictor
)

In [10]:
# the output of the cell is omitted
scenario.run()

[21:25:47] INFO     runn. objective: 2.72, state est.: [  1000.      0.      0. 115000. 101000.],  ]8;id=881104;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=10059;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    observation: [   -0.52     0.       0.   15000.    1000.  ], action: [0.],                     
                    value: 0.0027, time: 0.0000 (0.0%), episode: 1/1, iteration: 1/1                               

[21:25:54] INFO     runn. objective: 2.68, state est.: [  1000.        0.21      0.   106175.69    ]8;id=476593;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=592225;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    100000.  ], observation: [  -0.52    0.21    0.   6175.69    0.  ], action:                    
                    [0.], value: 0.0054, time: 0.0010 (10.0%), episode: 1/1, iteration: 1/1                        

[21:26:04] INFO     runn. objective: 2.68, state est.: [  1000.        0.36      0.   105809.      ]8;id=785607;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=545087;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    100000.  ], observation: [  -0.52    0.36    0.   5809.      0.  ], action:                    
                    [0.], value: 0.0081, time: 0.0020 (20.0%), episode: 1/1, iteration: 1/1                        

[21:26:16] INFO     runn. objective: 2.67, state est.: [  1000.        0.        0.   105416.18    ]8;id=27637;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=943253;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    100000.  ], observation: [  -0.52    0.      0.   5416.18    0.  ], action:                    
                    [0.], value: 0.0108, time: 0.0030 (30.0%), episode: 1/1, iteration: 1/1                        

[21:26:30] INFO     runn. objective: 2.67, state est.: [  1000.        0.19      0.   105016.78    ]8;id=74895;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=825070;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    100000.  ], observation: [  -0.52    0.19    0.   5016.78    0.  ], action:                    
                    [0.], value: 0.0134, time: 0.0040 (40.0%), episode: 1/1, iteration: 1/1                        

[21:26:46] INFO     runn. objective: 2.67, state est.: [  1000.01      0.52      0.   104548.54    ]8;id=623803;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=637320;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    100000.  ], observation: [  -0.52    0.52    0.   4548.54    0.  ], action:                    
                    [0.], value: 0.0161, time: 0.0050 (50.0%), episode: 1/1, iteration: 1/1                        

[21:27:04] INFO     runn. objective: 2.67, state est.: [  1000.01      0.4       0.   104140.26    ]8;id=545845;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=250367;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    100000.  ], observation: [  -0.52    0.4     0.   4140.26    0.  ], action:                    
                    [0.], value: 0.0188, time: 0.0060 (60.0%), episode: 1/1, iteration: 1/1                        

[21:27:25] INFO     runn. objective: 2.67, state est.: [  1000.01      0.11      0.   103834.15    ]8;id=436661;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=607441;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    100000.  ], observation: [  -0.52    0.11    0.   3834.15    0.  ], action:                    
                    [0.], value: 0.0214, time: 0.0070 (70.0%), episode: 1/1, iteration: 1/1                        

KeyboardInterrupt: 

ДЛЯ РАЗНЫХ ACTION ЗАКИДЫВАТЬ TORCH.TENSOR - BACKWARD, FORWARD.. в MPCObjective

Можно написать torch-оптимизатор, посмотреть на график оптимизации.

График Loss. Optimizer-step